In [54]:
import folium
from SaveEcoBotFinder import SaveEcoBotFinder
from IdMatcher import IdMatcher
from CamsWrapper import CamsWrapper


In [55]:
MATCHER = IdMatcher("співставлення ID постів з ID Cams.csv")
FINDER = SaveEcoBotFinder("Ecobot_16_11_2020")
m = folium.Map(location=[50.41671, 30.618716], zoom_start=12)

for id in MATCHER.byCamsId(6564):
    print(id)

    post = FINDER.getById(id)
    if post:
        value = post.getValue("2020-10-13", 12) # row №7822
        print(value)
        coord = MATCHER.getCoordinates(id)

        if value:
            folium.Circle(location=coord,
                          radius=10 * abs(value - 28.9),
                          popup=value).add_to(m)
        else:
            folium.Marker(coord,
                          icon=folium.Icon(color='red', icon='info-sign'),
                          popup=value).add_to(m)

m

43
Ecobot_16_11_2020/saveecobot_43.csv doesn't exits
968
15.35
1069
0
1105
0
1107
0
1115
0
1126
0
1127
0
1128
10.22
1193
21.0
1353
32.0
1415
0
1544
0
2762
0
2781
31.7
2809
0
3070
30.19
3231
11.94
3401
0
3497
Ecobot_16_11_2020/saveecobot_3497.csv doesn't exits
3504
31.75
3572
0
3620
31.6
3658
35.47
3678
15.92
3684
9.19
3685
8.54
4139
Ecobot_16_11_2020/saveecobot_4139.csv doesn't exits
4211
Ecobot_16_11_2020/saveecobot_4211.csv doesn't exits
12905
Ecobot_16_11_2020/saveecobot_12905.csv doesn't exits
12914
Ecobot_16_11_2020/saveecobot_12914.csv doesn't exits
12925
Ecobot_16_11_2020/saveecobot_12925.csv doesn't exits
12931
Ecobot_16_11_2020/saveecobot_12931.csv doesn't exits
12932
Ecobot_16_11_2020/saveecobot_12932.csv doesn't exits
12933
Ecobot_16_11_2020/saveecobot_12933.csv doesn't exits
12954
Ecobot_16_11_2020/saveecobot_12954.csv doesn't exits
12991
Ecobot_16_11_2020/saveecobot_12991.csv doesn't exits
13368
Ecobot_16_11_2020/saveecobot_13368.csv doesn't exits
13422
Ecobot_16_11_2020/s